In [ ]:
%matplotlib notebook
import pickle
import numpy as np
import math
import sk_geo_tools as sk
import matplotlib.pyplot as plt
import pg_fitter_tools as fit

In [ ]:
# Load fit results
#with open('../../PhotogrammetryAnalysis_msekatchev/results/full_ring/SK_ring.pkl', 'rb') as input:
#with open('results/SK_demo3.pkl', 'rb') as input:
with open('../../PhotogrammetryAnalysis/results/SK_demo5.pkl', 'rb') as input:    
    fitter_all = pickle.load(input)

reco_raw = {f: fitter_all.reco_locations[i] for f, i in fitter_all.feature_index.items()}
expected_locations = {f: fitter_all.seed_feature_locations[i] for f, i in fitter_all.feature_index.items()}

# Kabsch transform
errors, reco_transformed, scale, R, translation, location_mean = fit.kabsch_errors(expected_locations, reco_raw)
reco_locations = {f: reco_transformed[i] for f, i in fitter_all.feature_index.items()}

In [ ]:
from enum import IntEnum
class Coord(IntEnum):
    X = 0
    Y = 1
    Z = 2
    Total = 3
    Radial = 4
    Tangential = 5
    ProjectionX = 6    # Warning: These must be in same order as above for next function
    ProjectionY = 7
    ProjectionZ = 8
    ProjectionTotal = 9
    ProjectionRadial = 10
    ProjectionTangential = 11

In [ ]:
def calculate_distances(reco, exp, distarr, StartCoord=0):
    
    dist = 0

    for coord in range(0, 3):
        # X, Y, Z components
        distarr[StartCoord+coord].append(reco[coord]-exp[coord])
        # Total distance
        dist += (reco[coord]-exp[coord])**2
                
    dist = math.sqrt(dist)
    distarr[StartCoord+Coord.Total].append(dist)
    
    # Projections
    vector = reco - exp

    # Assume expected position as center of local cylindrical coordinate system
    radius = math.sqrt(exp[0]**2 + exp[1]**2)

    # Normal radial and tangential vectors
    radial_vector = (exp[0]/radius, exp[1]/radius, 0)
    tangential_vector = (exp[1]/radius, -exp[0]/radius, 0)
    
    # Calculate shifts along vectors
    radial_shift = np.dot(vector, radial_vector)
    tangential_shift = np.dot(vector, tangential_vector)

    distarr[StartCoord+Coord.Radial].append(radial_shift)
    distarr[StartCoord+Coord.Tangential].append(tangential_shift)
    
    #print("Rec.:", reco)
    #print("Exp.:", exp)
    #print("Dif. Vector:", vector)

    #print ("Rad. Vector:", radial_vector)
    #print ("Rad. Shift :", radial_shift)

    #print ("Tan. Vector:", tangential_vector)
    #print ("Tan. Shift :", tangential_shift)
    
    #print("PMT shift:", dist)

In [ ]:
def project_on_bolt_plane(pmt_center, bolt_center, bolt_normal):

    # Vector between dynode and bolt-ring center
    vector = pmt_center - bolt_center

    # Projection of dynode along normal to bolt-plane
    projected_center = pmt_center - np.dot(vector, bolt_normal) * bolt_normal

    return projected_center

In [ ]:
def get_bolt_center(chosenPMT, locations):
    
    bolts = []
    
    for item in locations:
        if(chosenPMT[:-3] == item[:-3] and item[-2:] != "00"):
            bolts.append((locations[item][0], locations[item][1], locations[item][2]))
    
    # No bolts found
    if not bolts:
        return None, None
    
    bolts_array = np.stack(list(bolts))
    average_bolt, normal = sk.fit_plane(bolts_array)
    return average_bolt, normal

In [ ]:
labelled = False
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')

pmt_shift_distance_array = [[] for i in range(len(Coord))]
bolt_shift_distance_array = [[] for i in range(len(Coord))]

for chosenPMT in reco_locations.keys():
    if (chosenPMT[-2:] != "00"):
        continue
        
    #print("PMT ID:", chosenPMT)
    
    # Get center point for this PMT
    expected_3D_dynode_centre = expected_locations[chosenPMT]
    reconstructed_3D_dynode_centre = reco_locations[chosenPMT]
    calculate_distances(reconstructed_3D_dynode_centre, expected_3D_dynode_centre, pmt_shift_distance_array)

    # Get bolt center for this PMT
    expected_3D_bolt_centre, expected_normal = get_bolt_center(chosenPMT, expected_locations)
    reconstructed_3D_bolt_centre, reconstructed_normal = get_bolt_center(chosenPMT, reco_locations)
    
    # Some datasets (e.g. March 2021 Barrel Ring) do not have detected bolts
    if expected_3D_bolt_centre is not None and reconstructed_3D_bolt_centre is not None:
        calculate_distances(reconstructed_3D_bolt_centre, expected_3D_bolt_centre, bolt_shift_distance_array)
    
        # Dynode projection onto bolt plane
        projected_reconstructed_3D_dynode_centre = project_on_bolt_plane(reconstructed_3D_dynode_centre, 
                                                                         reconstructed_3D_bolt_centre,
                                                                         reconstructed_normal)
        calculate_distances(projected_reconstructed_3D_dynode_centre, 
                            reconstructed_3D_bolt_centre, pmt_shift_distance_array, Coord.ProjectionX)
    
    #print("Rec. Dynode:", reconstructed_3D_dynode_centre)
    print("\n")
    
    if labelled == False:
        ax.scatter(reconstructed_3D_dynode_centre[0],reconstructed_3D_dynode_centre[1],reconstructed_3D_dynode_centre[2], color="red", marker = '*', label = "reconstructed dynode point")
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='red')

        ax.scatter(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], color="green", marker = '*', label = "expected dynode point")
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='green')
        labelled = True
    else:
        ax.scatter(reconstructed_3D_dynode_centre[0],reconstructed_3D_dynode_centre[1],reconstructed_3D_dynode_centre[2], color="red", marker = '*')
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='red')

        ax.scatter(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], color="green", marker = '*')
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='green')
    
    ax.text(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], chosenPMT[:-3],size=8, zorder=4, color='k')
    #title = str(chosenPMT)+", distance = "+str(round(pmt_shift_distance,3))+" px"
    #print(str(chosenPMT)+", distance = "+str(round(pmt_shift_distance,3))+" px\n")
    #plt.title(title)
    
plt.legend(loc=0)
plt.show()

In [ ]:
nbins = round(len(pmt_shift_distance_array[0])/4)

figuredir = "figures/"
import os
os.mkdir(figuredir)

def plot(axis_name, distarr):
    errorAverage = sum(distarr) / len(distarr)
    fig = plt.figure(figsize=(12,12))
    n, bins, patches = plt.hist(distarr, nbins, facecolor='g')
    plt.xlabel(axis_name+' Distance Error (cm)', size=25)
    plt.ylabel('Frequency', size=25)

    STD = np.std(distarr)
    STD_error = STD / np.sqrt(len(distarr))
    
    printSTD = "SD = " + str(round(STD,1)) + " cm"
    printAverage = "AVG = " + str(round(errorAverage,2)) + " ± " + str(round(STD_error,2)) + " cm"

    plt.title(printAverage+", "+printSTD, size=30, wrap=True)

    plt.tick_params(axis='both', labelsize=20)
    plt.grid(True)
    plt.show()
    plt.savefig(figuredir+"error-"+axis_name+".png")

In [ ]:
for coord in Coord:
    if len(pmt_shift_distance_array[coord]) != 0:
        plot("pmt_"+coord.name, pmt_shift_distance_array[coord])
    
    if len(bolt_shift_distance_array[coord]) != 0:
        plot("bolt_"+coord.name, bolt_shift_distance_array[coord])